***SHEET 5***


CARLOS FAROUK ABDALÁ RINCÓN

TOMAS CANDEO MONTOYA

GABRIEL DAVID JIMÉNEZ MOSQUERA

In [53]:
#IMPORTS
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
#import requests as rq
import itertools
from scipy.spatial.distance import pdist, squareform

### UTILS

In [54]:
def kMeans(D,k,eps=0.01,mu=None,max_iter=20):
 d=D.shape[1]
 if mu is None:
  mu=np.random.rand(k,d)*(np.max(D,axis=0)-np.min(D,axis=0))+np.min(D,axis=0)
 if type(mu)==list:
  mu=np.array(mu)
 s=False
 g=0
 while not s:
  C=[np.argmin([np.linalg.norm(mu[j]-x)for j in range(k)if not any(np.isnan(mu[j]))])for x in D]
  n=np.zeros(mu.shape)
  s=True
  for i in range(k):
   J=[D[j]for j in range(len(D))if C[j]==i]
   if len(J)>0:
    n[i]=np.mean(J,axis=0)
   else:
    n[i]=np.random.rand(1,d)*(np.max(D,axis=0)-np.min(D,axis=0))+np.min(D,axis=0)
   if np.linalg.norm(mu[i]-n[i])>eps:
    s=False
  mu=n
  g+=1
  if g>=max_iter:
   s=True
 return C,mu

### EXERCISE 1

#### EXERCISE 1.1

In [55]:
def getDistances(A):  
    pairwise_distances = pdist(A, metric='euclidean')
    distance_matrix = squareform(pairwise_distances)
    return distance_matrix

#### EXERCISE 1.2

In [56]:
def getW(D,U,V):
    acum_sum=0
    for _,index_u in enumerate(U):
        for _,index_v in enumerate(V):
            acum_sum+=(D[index_u][index_v])
    return acum_sum


#### EXERCISE 1.3

In [57]:
def getWIn(D, C):
    n = len(C)
    acum_sum = 0
    for i in range(n):
        for j in range(n):
            if C[i] == C[j]:
                acum_sum += D[i][j]
    return acum_sum/2

In [58]:
def getWOut(D, C):
    n = len(C)
    acum_sum = 0
    for i in range(n):
        for j in range(n):
            if C[i] != C[j]:
                acum_sum += D[i][j]
    return acum_sum/2

In [59]:
def getNIn(C):
    n = len(C)
    acum_sum = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                if C[i] == C[j]:
                    acum_sum += 1
    return acum_sum/2
    

In [60]:
def getNOut(C):
    n = len(C)
    acum_sum = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                if C[i] != C[j]:
                    acum_sum += 1
    return acum_sum/2

In [61]:
# Test weight

import pandas as pd_test
import numpy as np_test

dfIrisTest = pd_test.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")
A_Iris_Test = dfIrisTest[dfIrisTest.columns[:4]].astype(float)
C_Iris_Test = dfIrisTest[dfIrisTest.columns[4]]
print(type(A_Iris_Test))
D_Iris_Test = getDistances(A_Iris_Test)
clusters = np.where(C_Iris_Test == 'setosa')[0]
C2 = np.where(C_Iris_Test == 'versicolor')[0]
C3 = np.where(C_Iris_Test == 'virginica')[0]
CList = [clusters, C2, C3]

print ("\nTest Weight Measures\n------------------")
expectedW = {
    (0,1): 8246,
    (0,2): 12056,
    (1,2): 4606
}
wSummary = "W: "
wFailed = False
for i in range(3):
    for j in range(i):
        p1 = (i,j)
        p2 = (j,i)
        W1 = np_test.round(getW(D_Iris_Test, CList[i], CList[j]))
        W2 = np_test.round(getW(D_Iris_Test, CList[j], CList[i]))
        if W1 != W2:
            if not wFailed:
                wSummary += "failed"
            wSummary += "\n\tasymmetry of W: " + str(W1) + " != " + str(W2)
            wFailed = True
        
        if W1 != expectedW[p2]:
            if not wFailed:
                wSummary += "failed"
            wSummary += "\n\tunexpected value of W: " + str(W1) + " instead of expected " + str(expectedW[p2])
            wFailed = True
if not wFailed:
    wSummary += "ok"
print(wSummary)
expectedWIn = 3518
expectedWOut = 24908
expectedNIn = 3675
expectedNOut = 7500
print("WIn:", "ok" if np_test.abs(np_test.round(getWIn(D_Iris_Test, C_Iris_Test)) - expectedWIn) < 2 else "failed")
print("WOut:", "ok" if np_test.abs(np_test.round(getWOut(D_Iris_Test, C_Iris_Test)) - expectedWOut) < 2 else "failed")
print("NIn:", "ok" if getNIn(C_Iris_Test) == 3675 else "failed")
print("NOut:", "ok" if getNOut(C_Iris_Test) == 7500 else "failed")

<class 'pandas.core.frame.DataFrame'>

Test Weight Measures
------------------
W: ok
WIn: ok
WOut: ok
NIn: ok
NOut: ok


#### EXERCISE 1.4

In [62]:
def getBetaCV(A,C):
    D=getDistances(A)
    W_in=getWIn(D,C)
    N_in=getNIn(C)
    W_out=getWOut(D,C)
    N_out=getNOut(C)
    betaCV_value= (W_in/N_in)/(W_out/N_out)
    return betaCV_value


In [63]:
def getCIndex(A,C):
    D=getDistances(A)
    W_in=getWIn(D,C)
    N_in=getNIn(C)
    values=[]
    rows,columns=D.shape
    for index_row in range(rows):
        for index_column in range(columns):
            if index_row<index_column:
                values.append(D[index_row][index_column])
    values_order_upward=np.sort(values)
    values_order_downward=values_order_upward[::-1]
    W_min=0
    W_max=0
    for index in range(int(N_in)):
        W_min+=values_order_upward[index]
        W_max+=values_order_downward[index]
    C_index=(W_in-W_min)/(W_max-W_min)
    return C_index


In [64]:
def getNormalizedCut(A, C):
    k=len(set(C))
    D=getDistances(A)
    categories=np.unique(C)
    normalizedCut=0
    index_groups = [[] for _ in range(k)]
    for index in range(len(C)):
        for index_category,category in enumerate(categories):
            if(C[index]==category):
                index_groups[index_category].append(index)

    for index_cluster in range(len(index_groups)):
        rest_index=[]
        for index_rest in range (len(index_groups)):
            if (index_cluster!=index_rest):
                rest_index=rest_index+index_groups[index_rest]
        value_up=getW(D,index_groups[index_cluster],rest_index)
        value_down=getW(D,index_groups[index_cluster],index_groups[index_cluster])+value_up
        normalizedCut+=(value_up/value_down)

    return normalizedCut

In [65]:
def getDunn(A,C):
    D = getDistances(A)
    n = len(C)
    min_out = float('inf')
    max_in = 0
    for i in range(n):
        for j in range(n):
            if C[i] != C[j]:
                out_distance = D[i][j]
                if out_distance < min_out:
                    min_out = out_distance
    for i in range(n):
        for j in range(n):
            if C[i] == C[j]:
                in_distance = D[i][j]
                if in_distance > max_in:
                    max_in = in_distance
    dunn = min_out / max_in
    return dunn

In [78]:
def getDaviesBouldin(A, C):
    
    return 0
    

In [142]:
def getSilhouette(A,C):
    D = getDistances(A)
    n = len(C)
    categories = np.unique(C)
    print(categories)
    silhouette = 0
    acum_in = 0
    count_in = 0
    min_u_out = float('inf')
    for i in range(n):
        for j in range(n):
            if i != j:
                if C[i] == C[j]:
                    acum_in += D[i][j]
                    count_in += 1
        u_in = acum_in / (count_in - 1)
        for category in categories:
            if category != C[i]:
                acum_out = 0
                count_out = 0
                for j in range(n):
                    if C[j] == category:
                        acum_out += D[i][j]
                        count_out += 1
                u_out = acum_out / count_out
                if u_out < min_u_out:
                    min_u_out = u_out
        silhouette += (min_u_out - u_in) / max(min_u_out, u_in)
    silhouette = silhouette / n
    return silhouette


In [140]:
'''
import numpy as np

def getSilhouette(A, C):
    # Compute pairwise distance matrix
    D = getDistances(A)
    n_samples = len(C)
    silhouette_vals = np.zeros(n_samples)
    for i in range(n_samples):
        label_i = C[i]
        dists_i = D[i]

        # Compute the mean distance of the current sample to other samples in the same cluster
        intra_cluster_mean_dist = np.mean(dists_i[C == label_i])

        # Compute the mean distance of the current sample to samples in other clusters
        min_inter_cluster_mean_dist = np.min([np.mean(dists_i[C == label_j]) for label_j in set(C) if label_j != label_i])

        # Compute silhouette value for the current sample
        silhouette_vals[i] = (min_inter_cluster_mean_dist - intra_cluster_mean_dist) / max(intra_cluster_mean_dist, min_inter_cluster_mean_dist)

    # Compute mean silhouette score
    silhouette_score = np.mean(silhouette_vals)
    
    return silhouette_score
'''


'\nimport numpy as np\n\ndef getSilhouette(A, C):\n    # Compute pairwise distance matrix\n    D = getDistances(A)\n    n_samples = len(C)\n    silhouette_vals = np.zeros(n_samples)\n    for i in range(n_samples):\n        label_i = C[i]\n        dists_i = D[i]\n\n        # Compute the mean distance of the current sample to other samples in the same cluster\n        intra_cluster_mean_dist = np.mean(dists_i[C == label_i])\n\n        # Compute the mean distance of the current sample to samples in other clusters\n        min_inter_cluster_mean_dist = np.min([np.mean(dists_i[C == label_j]) for label_j in set(C) if label_j != label_i])\n\n        # Compute silhouette value for the current sample\n        silhouette_vals[i] = (min_inter_cluster_mean_dist - intra_cluster_mean_dist) / max(intra_cluster_mean_dist, min_inter_cluster_mean_dist)\n\n    # Compute mean silhouette score\n    silhouette_score = np.mean(silhouette_vals)\n    \n    return silhouette_score\n'

In [121]:
def getMetric(A, C, metric):
    if metric == "beta":
        return getBetaCV(A, C)
    elif metric == "cindex":
        return getCIndex(A, C)
    elif metric == "nc":
        return getNormalizedCut(A, C)
    elif metric == "dunn":
        return getDunn(A, C)
    elif metric == "db":
        return getDaviesBouldin(A, C)
    elif metric == "sil":
        return getSilhouette(A, C)
    else:
        return "metrica invalida"

In [143]:
# Test weights
import pandas as pd_test
import numpy as np_test
dfIrisTest = pd_test.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")
A_Iris_Test = dfIrisTest[dfIrisTest.columns[:4]].astype(float)
C_Iris_Test = dfIrisTest[dfIrisTest.columns[4]]
D_Iris_Test = getDistances(A_Iris_Test)

# Test metrics
print ("\nTest Metrics\n------------------")
expected = {
    "beta": 0.2882861014913346,
    "cindex": 0.046803774122703735,
    "nc": 2.6150343040385264,
    "dunn": 0.05848053214719304,
    "db": 0.8445815484442534,
    "sil": 0.5032506980665507
}
for m in expected:
    e = np.round(expected[m], 2)
    a = getMetric(A_Iris_Test, C_Iris_Test, m)
    a = np.round(a, 2) if not a is None else None
    print(m + ":\t", "ok" if e == a else "failed. Expected " + str(e) + " but saw " + str(a))


Test Metrics
------------------
beta:	 ok
cindex:	 ok
nc:	 ok
dunn:	 ok
db:	 failed. Expected 0.84 but saw 0
['setosa' 'versicolor' 'virginica']
sil:	 failed. Expected 0.5 but saw 0.39
